In [34]:
#CARREGA A(S) BIBLIOTECAS
import pandas as pd
import json

In [35]:
#CARREGA O ARQUIVO QUE CONTEM A LOCALIZAÇÃO DAS PLANTAS DE PRODUCAO DE ÓLEO LUBRIFICANTE EM SOLO BRASILEIRO
plantas_producao = pd.read_excel("../data/boletim_1/Anexo_E.xlsx", sheetname="30-Mapa_Planta_Prod_OLAC")
plantas_producao.head()

,CNPJ,Produtor,Município,UF
0,57941890000153,AGECOM PRODUTOS DE PETRÓLEO LTDA.,MAUA,SP
1,3439153000128,BIOLUB QUÍMICA LTDA -ME,SOROCABA,SP
2,33194978000190,CASTROL BRASIL LTDA,RIO DE JANEIRO,RJ
3,45036670000104,CHEMLUB PRODUTOS QUÍMICOS LTDA,VALINHOS,SP
4,5524572001084,CHEVRON BRASIL LUBRIFICANTES LTDA.,DUQUE DE CAXIAS,RJ


In [36]:
#CRIA UM DICIONÁRIO PARA ARMAZENAR O UF DAS PLANTAS DE PRODUCAO
empresa_prod_uf = {}

for index,row in plantas_producao.iterrows():
    empresa = row['Produtor']
    if empresa not in empresa_prod_uf:
        empresa_prod_uf[empresa] = row['UF']
    else:
    #CASO UMA EMPRESA TENHA MAIS DE UMA PLANTA INDUSTRIAL, IMPRIMIR O NOME DA EMPRESA E A UF ONDE A PLANTA ESTÁ.    
        print(empresa+" - "+row['UF']) # A IPIRANGA NESSE CASO POSSUI DUAS PLANTAS NO MESMO ESTADO (RJ)
        continue

IPIRANGA PRODUTOS DE PETRÓLEO S.A - RJ


In [37]:
#VENDAS NACIONAIS DE OLAC PELOS PRODUTORES E IMPORTADORES
#A COLUNA PRODUTOR-IMPORTADOR POSSUI A INFORMAÇÃÕ DE UMA EMPRESA QUE DECLARA UMA OPERAÇÃO DE COMERCIALIZAÇÃO À ANP.
#TODAVIA, ESSE PRODUTO COMERCIALIZADO PODE TER A SEGUINTE ORIGEM:
### 1. SER PRODUZIDO EM PLANTA INDUSTRIAL PRÓPRIA EM SOLO BRASILEIRO.
### 2. SER PRODUZIDO EM PLANTA INDUSTRIAL DE TERCEIRO EM SOLO BRASILEIRO.
### 3. SER PRODUZID FORA DO BRASIL (PLANTA PRÓPRIA OU NÃO.)
vendas = pd.read_excel("../data/boletim_1/Anexo_H.xlsx", sheetname="41-Vendas OLAC Geral")
vendas.head(5)

,Data,Cód Regulado ANP,Produtor-Importador,Código do Produto,Descrição do Produto,Região do Destinatário,UF do Destinatário,Volume-Litros
0,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUL,PR,9828
1,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUL,SC,800
2,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620501001,Ciclo Otto,SUL,PR,3685
3,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620501001,Ciclo Otto,SUL,RS,480
4,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620501001,Ciclo Otto,SUL,SC,5040


In [38]:
#TROCAR O NOME DA EMPRESA (PRODUTOR-IMPORTADOR) PELO UF ONDE ESTÁ LOCALIZADO A PLANTA DE PRODUÇÃO.
vendas['Produtor-Importador'] = vendas['Produtor-Importador'].map(empresa_prod_uf)

In [39]:
#NÚMERO DE REGISTROS DE DECLARAÇÃO DE COMERCIALIZAÇÃO DE EMPRESAS QUE NÃO PRODUZIRAM EM PLANTA PRÓPRIA NO BRASIL OU SÃO 
#CONSIDERADAS IMPORTADORAS
vendas['Produtor-Importador'].isnull().sum()

9848

In [40]:
#PERCENTUAL DE REGISTROS DE DECLARAÇÃO DE COMERCIALIZAÇÃO QUE SÃO DE EMPRESAS QUE NÃO PRODUZIRAM EM PLANTA PRÓPRIA NO BRASIL OU SÃO 
#CONSIDERADAS IMPORTADORAS
str(round(vendas['Produtor-Importador'].isnull().sum()/vendas['Produtor-Importador'].shape[0]*100,2))+"%"

'26.14%'

In [41]:
#SOMA DOS VOLUMES DE COMERCIALIZACAO DE EMPRESAS DETENTORAS & PRODUTORAS
count_volume_detentor_produtor = 0

#SOMA DOS VOLUMES DE EMPRESAS QUE SÃO DETENTORAS/TERCERIZADORAS OU DETENTORAS/IMPORTADORAS
count_volume_outros = 0
for index,row in vendas.iterrows():
    if type(row['Produtor-Importador']) != str:
        count_volume_outros += row['Volume-Litros']
    else:
        count_volume_detentor_produtor += row['Volume-Litros']

In [42]:
print("Volume total de empresas Detentoras/Produtoras "+str(count_volume_detentor_produtor/1000)+ " m³")
print("Volume total de empresas Detentoras/Tercerizadoras/Importadoras "+str(count_volume_outros/1000)+ " m³")

Volume total de empresas Detentoras/Produtoras 1131775.986 m³
Volume total de empresas Detentoras/Tercerizadoras/Importadoras 97916.397 m³


In [43]:
#PERCENTUAL EM VOLUME DOS REGISTROS QUE NÃO PODE SE INFERIR QUAL A ORIGEM DA PRODUÇÃO, APENAS O DESTINATÁRIO.
str(round(count_volume_outros/(count_volume_outros+count_volume_detentor_produtor)*100,2))+"%"

'7.96%'

In [44]:
#LISTA PARA ABRIGAR O INDEX DOS REGISTROS QUE NÃO POSSUEM A IDENTIFICAÇÃO DA UF DA ORIGEM DA PRODUÇÃO
list_index_nan = []
for index,row in vendas.iterrows():
    if type(row['Produtor-Importador']) != str:
        list_index_nan.append(index)

In [45]:
#DELETAR DO DATAFRAME DOS OS REGISTROS QUE NÃO POSSUEM A IDENTIFICAÇÃO DA UF DA ORIGEM DA PRODUÇÃO
vendas_uf = vendas[vendas['Produtor-Importador'].notnull()]

In [46]:
#RESETAR O INDEX DO DATAFRAME
vendas_uf.reset_index(inplace=True)
del vendas_uf['index']

In [47]:
vendas_uf.columns = ['Data', 'Cód Regulado ANP', 'UF_origem',
       'Código do Produto', 'Descrição do Produto', 'Região do Destinatário',
       'UF_destino', 'Volume-Litros']

In [48]:
#CRIA UMA LISTA COM TODAS AS UF'S
lista_UF = vendas_uf['UF_destino'].unique().tolist()

#ORGANIZA A LISTA POR ORDEM ALFABÉTICA
lista_UF.sort()

In [49]:
#LISTA DE DICIONÁRIOS
lista_dict_uf = []
for uf in lista_UF:
    dict_uf = {"name": uf}
    lista_dict_uf.append(dict_uf)

In [50]:
#CRIA UM DICIONÁRIO PARA CODIFICAR AS UF'S
uf_codigo = {}
count = 0

for uf in lista_UF:
    uf_codigo[uf] = count
    count += 1

In [51]:
#CODIFICA AS COLUNAS UTILIZANDO O DICIONÁRIO uf_codigo
vendas_uf['UF_origem'] = vendas_uf['UF_origem'].map(uf_codigo)
vendas_uf['UF_destino'] = vendas_uf['UF_destino'].map(uf_codigo)

C:\Users\ferraz\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\ferraz\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [52]:
#VERIFICACÃO DA CODIFICACAO
vendas_uf.head()

,Data,Cód Regulado ANP,UF_origem,Código do Produto,Descrição do Produto,Região do Destinatário,UF_destino,Volume-Litros
0,12/2016,200296,17,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUL,17,9828
1,12/2016,200296,17,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUL,23,800
2,12/2016,200296,17,620501001,Ciclo Otto,SUL,17,3685
3,12/2016,200296,17,620501001,Ciclo Otto,SUL,22,480
4,12/2016,200296,17,620501001,Ciclo Otto,SUL,23,5040


In [53]:
#CRIAR UMA LISTA DE DICIONÁRIOS PARA RECEBER OS DADOS DE LIGAÇÃO ENTRE OS NÓS
lista_dict_links = []
for index,row in vendas_uf.iterrows():
    dict_links = {"source": row['UF_origem'], "target": row['UF_destino'], "value": row['Volume-Litros']/1000}
    lista_dict_links.append(dict_links)

In [54]:
#CRIAR UM OBJETO PARA CRIAR O DIAGRAMA DE SANKEY
data_vendas_olac = { "nodes" : lista_dict_uf , "links" : lista_dict_links }

In [55]:
data_vendas_olac

{'links': [{'source': 17, 'target': 17, 'value': 9.828},
  {'source': 17, 'target': 23, 'value': 0.8},
  {'source': 17, 'target': 17, 'value': 3.685},
  {'source': 17, 'target': 22, 'value': 0.48},
  {'source': 17, 'target': 23, 'value': 5.04},
  {'source': 17, 'target': 17, 'value': 1.184},
  {'source': 17, 'target': 17, 'value': 17.764},
  {'source': 17, 'target': 22, 'value': 13.8},
  {'source': 17, 'target': 23, 'value': 71.772},
  {'source': 17, 'target': 17, 'value': 30.519},
  {'source': 17, 'target': 23, 'value': 2.2},
  {'source': 25, 'target': 25, 'value': 7.22},
  {'source': 25, 'target': 25, 'value': 117.4},
  {'source': 25, 'target': 8, 'value': 180.0},
  {'source': 25, 'target': 25, 'value': 15.84},
  {'source': 17, 'target': 6, 'value': 29.28},
  {'source': 17, 'target': 8, 'value': 15.72},
  {'source': 17, 'target': 4, 'value': 9.6},
  {'source': 17, 'target': 5, 'value': 4.56},
  {'source': 17, 'target': 9, 'value': 1.92},
  {'source': 17, 'target': 14, 'value': 13.16}

In [56]:
#CRIA UMA LISTA COM A IDENTIFICACAO DOS ESTADOS DE ORIGEM E DE DESTINO
lista_source = []
lista_target = []
for data in data_vendas_olac['links']:
    if data['source'] not in lista_source:
        lista_source.append(data['source'])
    if data['target'] not in lista_target:
        lista_target.append(data['target'])


In [57]:
#CRIA UM DICIONÁRIO ONDE IRÁ RECEBER A INFORMAÇÃO AGREGADA DAS MOVIMENTAÇÕES.
#O OBJETIVO É TER UM REGISTRO ÚNICO DE MOVIMENTAÇÃO (SOMÁTORIO) ENTRE DUAS ENTIDADES
nome_dict = []
for UFcod in lista_source:
    name = UFcod
    nome_dict.append(name)
dict_list_source={ name:{} for name in nome_dict}

In [58]:
#ALIMENTA O DICIONÁRIO COM OS DADOS AGREGADOS
for data in data_vendas_olac['links']:
    for source in dict_list_source:
        if data['source'] == source:
            if data['target'] not in dict_list_source[source]:
                dict_list_source[source][data['target']] = data['value']
            else:
                dict_list_source[source][data['target']] += data['value']   
    

In [59]:
#VISUALIZAR A ESTRUTURA DO DICIONÁRIO CRIADO
dict_list_source

{2: {0: 44.217,
  2: 610.8130000000001,
  13: 4.191999999999999,
  20: 520.4379999999998,
  21: 224.457},
 4: {1: 0.28800000000000003,
  2: 0.36,
  4: 1576.3029999999999,
  5: 231.547,
  7: 0.12,
  8: 0.41600000000000004,
  10: 63.17300000000001,
  11: 1.49,
  13: 0.24,
  14: 1.06,
  16: 307.6219999999999,
  17: 0.292,
  18: 4.8,
  19: 114.89999999999999},
 10: {0: 287.52699999999993,
  1: 857.9479999999998,
  2: 863.4159999999999,
  3: 114.70400000000001,
  4: 5082.051999999998,
  5: 1933.5579999999998,
  6: 2022.5149999999994,
  7: 2755.7530000000006,
  8: 4616.857999999999,
  9: 1987.3970000000002,
  10: 26391.53900000001,
  11: 2829.555,
  12: 5589.155999999997,
  13: 2862.358,
  14: 1178.8119999999997,
  15: 3279.7339999999986,
  16: 970.1560000000002,
  17: 8892.038999999999,
  18: 5355.512,
  19: 1281.716,
  20: 867.8509999999998,
  21: 220.85199999999998,
  22: 6174.731000000001,
  23: 4896.386000000002,
  24: 499.352,
  25: 25148.473999999984,
  26: 816.056},
 12: {8: 1.0,
  1

In [60]:
#IMPRESSÃO DOS DADOS AGREGADOS
for key,values in dict_list_source.items():
    for val in values.items():
        print('source: '+str(key)+" => "+"target: "+str(val[0])+" => "+"value: "+str(val[1]))

source: 17 => target: 0 => value: 0.696
source: 17 => target: 1 => value: 11.672
source: 17 => target: 2 => value: 154.988
source: 17 => target: 3 => value: 89.464
source: 17 => target: 4 => value: 486.8859999999997
source: 17 => target: 5 => value: 317.768
source: 17 => target: 6 => value: 248.12499999999994
source: 17 => target: 7 => value: 107.10800000000006
source: 17 => target: 8 => value: 327.08000000000004
source: 17 => target: 9 => value: 313.16
source: 17 => target: 10 => value: 423.598
source: 17 => target: 11 => value: 213.166
source: 17 => target: 12 => value: 568.844
source: 17 => target: 13 => value: 180.298
source: 17 => target: 14 => value: 181.69200000000004
source: 17 => target: 15 => value: 439.73800000000006
source: 17 => target: 16 => value: 103.90800000000003
source: 17 => target: 17 => value: 3511.0610000000006
source: 17 => target: 18 => value: 192.024
source: 17 => target: 19 => value: 245.18800000000002
source: 17 => target: 20 => value: 373.134
source: 17 => 

In [61]:
#CRIAR UMA LISTA DE DICIONÁRIOS PARA RECEBER OS DADOS AGREGADOS DE LIGAÇÃO ENTRE OS NÓS
lista_dict_agg = []
for key,values in dict_list_source.items():
    for val in values.items():
        dict_links = {"source": key, "target": val[0], "value": round(val[1],3)}
        lista_dict_agg.append(dict_links)

In [62]:
lista_dict_agg

[{'source': 17, 'target': 0, 'value': 0.696},
 {'source': 17, 'target': 1, 'value': 11.672},
 {'source': 17, 'target': 2, 'value': 154.988},
 {'source': 17, 'target': 3, 'value': 89.464},
 {'source': 17, 'target': 4, 'value': 486.886},
 {'source': 17, 'target': 5, 'value': 317.768},
 {'source': 17, 'target': 6, 'value': 248.125},
 {'source': 17, 'target': 7, 'value': 107.108},
 {'source': 17, 'target': 8, 'value': 327.08},
 {'source': 17, 'target': 9, 'value': 313.16},
 {'source': 17, 'target': 10, 'value': 423.598},
 {'source': 17, 'target': 11, 'value': 213.166},
 {'source': 17, 'target': 12, 'value': 568.844},
 {'source': 17, 'target': 13, 'value': 180.298},
 {'source': 17, 'target': 14, 'value': 181.692},
 {'source': 17, 'target': 15, 'value': 439.738},
 {'source': 17, 'target': 16, 'value': 103.908},
 {'source': 17, 'target': 17, 'value': 3511.061},
 {'source': 17, 'target': 18, 'value': 192.024},
 {'source': 17, 'target': 19, 'value': 245.188},
 {'source': 17, 'target': 20, 'valu

In [63]:
#CRIAR UM OBJETO PARA CRIAR O DIAGRAMA DE SANKEY COM DADOS AGREGADOS
vendas_olac_agg = { "nodes" : lista_dict_uf , "links" : lista_dict_agg }

In [64]:
vendas_olac_agg 

{'links': [{'source': 17, 'target': 0, 'value': 0.696},
  {'source': 17, 'target': 1, 'value': 11.672},
  {'source': 17, 'target': 2, 'value': 154.988},
  {'source': 17, 'target': 3, 'value': 89.464},
  {'source': 17, 'target': 4, 'value': 486.886},
  {'source': 17, 'target': 5, 'value': 317.768},
  {'source': 17, 'target': 6, 'value': 248.125},
  {'source': 17, 'target': 7, 'value': 107.108},
  {'source': 17, 'target': 8, 'value': 327.08},
  {'source': 17, 'target': 9, 'value': 313.16},
  {'source': 17, 'target': 10, 'value': 423.598},
  {'source': 17, 'target': 11, 'value': 213.166},
  {'source': 17, 'target': 12, 'value': 568.844},
  {'source': 17, 'target': 13, 'value': 180.298},
  {'source': 17, 'target': 14, 'value': 181.692},
  {'source': 17, 'target': 15, 'value': 439.738},
  {'source': 17, 'target': 16, 'value': 103.908},
  {'source': 17, 'target': 17, 'value': 3511.061},
  {'source': 17, 'target': 18, 'value': 192.024},
  {'source': 17, 'target': 19, 'value': 245.188},
  {'so

In [65]:
with open('../data/vendas_olac_uf.json', 'w') as fp:
    json.dump(data_vendas_olac, fp)

In [66]:
with open('../data/vendas_olac_agg.json', 'w') as fp:
    json.dump(vendas_olac_agg, fp)